# Data Cleaning
In order to use this data for training a model for calibrtion offset, we want to make sure that the distributions of the data are correct and don't include any outliers that might skew the data. 

Similarly, there are some features that we don't want to use because the distributions seen in the raw machine data differ too greatly from those seen during training (e.g. trained with a constant value but distributions seen in reality, ranges outside of the training range).

There are also some features in the model that are combinations of multiple input PVs so we need to create those features and mappings ourselves. 

The goal of this notebook is to condense the dataframe of all of the PVs recorded down to a dataframe that can be used for training calibration layers.

To Do:
1. [x] subset data to include only relevant features
1. [x] remove unphysical values
1. [x] create compound features
1. [x] remove outliers (anything more or less than 3*std)
1. [x] drop any points where not all features are present
    1. *save here* incase we want to use the normal features as well
1. [?] filter points where beam size changes without features changing
1. [x] replace ignored features with default values (save as ref)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import json
import seaborn as sns
import glob
from plotting import plot_series, plot_boxplot, plot_histogram
from sklearn.linear_model import LinearRegression
from adtk.transformer import RegressionResidual
from utils import get_date_from_df
from datetime import datetime

%load_ext autoreload
%autoreload 2

In [ ]:
data = pd.read_pickle('archive_data/raw/injector_2021-11-16.pkl')

In [ ]:
# time_series = pd.read_pickle('archive_data\injector_2021-04-14.pkl')
# time_series = time_series['2022-09-23 12:00:00':'2022-09-23 23:59:59']
original_time_series = [pd.read_pickle(filename) for filename in glob.glob('archive_data/injector_*.pkl')]
all_time_series = [pd.read_pickle(filename) for filename in glob.glob('archive_data/injector_*.pkl')]
print(len(all_time_series))


In [ ]:
plot_series(all_time_series[0], show=True)

In [ ]:
all_time_series[0].head()

In [ ]:
all_time_series[0].columns

The dataframes will all contain both the actual and the control values for the quadrupole settings. For now the intention is to do all of the data **processing** using the CONTROL PVs (e.g. dropping duplicates etc) and then using the ACTUAL PVs for training the model.

In [ ]:
# for time_series in all_time_series:
#     # date = str(time_series.index[0].to_pydatetime().date())
#     fig, ax = plot_series(time_series)
#     fig.savefig(f'archive_data/data_exploration/time_series/{get_date_from_df(time_series)}.png')
#     plt.close(fig)

## Relevant PVs
Here we take the subset of PVs of interest from the full data

In [ ]:
ignored_features = [
    # 'IRIS:LR20:130:CONFG_SEL',  # out of range - swapping to use sigma_x and sigma_y instead
    'ACCL:IN20:400:L0B_ADES',  # out of range - trained as constant but varying at value outside of this
    'Pulse_length',  # not measured as PV - assume is fixed at reference
    'FBCK:BCI0:1:CHRG_S'  # trained as constant but measured a distribution
]

In [ ]:
with open('configs/pv_info.json', 'r') as f:
    pv_info = json.load(f)
    f.close()

with open('configs/model_info.json', 'r') as f:
    model_info = json.load(f)
    f.close()


In [ ]:
all_time_series[0].columns

In [ ]:
# this dictionary contains the mappings for the outputs as well as the inputs so we don't need to specify the outputs separately
relevant_pvs = []
for pv_name in list(pv_info['pv_name_to_sim_name'].keys()) + ['CAMR:IN20:186:YRMS', 'CAMR:IN20:186:XRMS']:
    if pv_name in all_time_series[0].columns:
        relevant_pvs.append(pv_name.replace('BCTRL', 'BACT'))
    # if 'BACT' in pv_name and pv_name.replace('BCTRL','BACT') in all_time_series[0].columns:
    #     # we want to include the actual and the control points just in case
    #     relevant_pvs.append(pv_name.replace('BCTRL','BACT'))

print(relevant_pvs)

In [ ]:
def drop_na_and_duplicates(time_series, relevant_pvs):
    # drop any NA rows
    time_series = time_series[relevant_pvs].dropna()
    # drop any duplicated columns
    time_series = time_series.loc[:,~time_series.columns.duplicated()]
    return time_series

In [ ]:
all_time_series = [drop_na_and_duplicates(time_series, relevant_pvs) for time_series in all_time_series]

## Compound Features

In [ ]:
def add_rdist(time_series):
    r_dist = np.sqrt(time_series['CAMR:IN20:186:XRMS'].values**2 + time_series['CAMR:IN20:186:YRMS'].values**2)
    time_series['CAMR:IN20:186:R_DIST'] = r_dist

    # once we have it we can drop these other two dolumns
    time_series = time_series.drop('CAMR:IN20:186:XRMS',axis=1)
    time_series = time_series.drop('CAMR:IN20:186:YRMS', axis=1)
    return time_series

In [ ]:
all_time_series = [add_rdist(time_series) for time_series in all_time_series]

## Remove Unphysical Values

In [ ]:
# measured_pvs = [pv for pv in relevant_pvs if pv not in ignored_features]
# print(measured_pvs)
# print(len(measured_pvs))
measured_pvs = [pvname for pvname in pv_info['pv_name_to_sim_name'].keys() if pvname in all_time_series[0].columns and pvname not in ignored_features]

In [ ]:
# here we define the ranges of the PVs that we're interested in
thresholds = {}
for pv in pv_info['pv_name_to_sim_name']:
    sim_name = pv_info['pv_name_to_sim_name'][pv]
    sim_to_pv_factor = pv_info['sim_to_pv_factor'][sim_name]
    if sim_name in model_info['loc_in']:
        sim_idx = model_info['loc_in'][sim_name]
        lower_bound = model_info['train_input_mins'][sim_idx] * sim_to_pv_factor
        upper_bound = model_info['train_input_maxs'][sim_idx] * sim_to_pv_factor
        if lower_bound != upper_bound:
            if lower_bound < upper_bound:
                thresholds[pv] = {
                    'lower': lower_bound,
                    'upper': upper_bound,
                }
            else:
                thresholds[pv] = {
                    'lower': upper_bound,
                    'upper': lower_bound,
                }


thresholds.update(
    {
        'OTRS:IN20:571:XRMS': {'lower': 0, 'upper': 1000},
        'OTRS:IN20:571:YRMS': {'lower': 0, 'upper': 1000},
        # 'CAMR:IN20:186:R_DIST': {'lower': 250, }
    }
)


In [ ]:
thresholds

In [ ]:
def remove_unphysical_and_ood_values(time_series, measured_pvs, thresholds):
    original_len = len(time_series)
    for pv in measured_pvs:
        if pv in thresholds.keys():
            time_series = time_series[(time_series[pv] > thresholds[pv].get('lower')) & (time_series[pv] < thresholds[pv].get('upper'))]

    final_len = len(time_series)
    print(f'Removed {original_len - final_len} points from time series of {original_len} points')
    return time_series

In [ ]:
all_time_series = [df for df in [remove_unphysical_and_ood_values(time_series, measured_pvs, thresholds) for time_series in all_time_series] if len(df > 0)]
# print(len(test))

## Remove Outliers
Here we inspect each of the histograms and determine what outlier threshold we should be using for each input feature (if any).

In [ ]:
# print(len(measured_pvs))
# print(measured_pvs)

In [ ]:
relevant_pvs

In [ ]:
measured_pvs = [pvname for pvname in pv_info['pv_name_to_sim_name'].keys() if pvname in all_time_series[0].columns and pvname not in ignored_features]

In [ ]:
measured_pvs
# 

In [ ]:
len(measured_pvs)

In [ ]:
for time_series in all_time_series:
    plot_boxplot(time_series, measured_pvs, show=False)

In [ ]:
def remove_outliers(df,columns,n_std):
    original_len = len(df)
    df = df.copy()
    for col in columns:       
        mean = df[col].mean()
        sd = df[col].std()
        
        df = df[(df[col] <= mean+(n_std*sd))]
    final_len = len(df)
    print(f'Removed {original_len - final_len} points from time series of {original_len} points')
    return df

In [ ]:
all_time_series =[df for df in [remove_outliers(time_series, measured_pvs, 3) for time_series in all_time_series] if len(df > 0)]
# time_series = remove_outliers(time_series, measured_pvs, 3)

In [ ]:
len(all_time_series)

In [ ]:
for time_series in all_time_series:
    plot_histogram(time_series, measured_pvs)

## Overwrite Ignored Values
For some values where the range is far outside that of the training data, we want to use the reference point value instead of the measured value.

In [ ]:
from configs.ref_config import ref_point
ref_point = ref_point[0]

In [ ]:
ignored_features

In [ ]:
def overwrite_ignored_features(time_series, model_info, pv_info, ref_point, ignored_features):
    for pvname in ignored_features:
        sim_name = pv_info['pv_name_to_sim_name'][pvname]
        # ref points are in sim values so we need to convert to pv_units
        feature_loc = model_info['loc_in'][sim_name]
        reference_val = pv_info['sim_to_pv_factor'][sim_name] * ref_point[feature_loc]
        time_series[pvname] = reference_val
        print(f'reset {pvname} to {reference_val}')
    return time_series


In [ ]:
all_time_series = [overwrite_ignored_features(time_series, model_info, pv_info, ref_point, ignored_features) for time_series in all_time_series]

In [ ]:
# time_series_subset = time_series_subset.copy()
ignored_outputs = ['sigma_z', 'norm_emit_x', 'norm_emit_y']

## Drop NAs
Here we drop the items in the dataframe where not all features are present

In [ ]:
all_time_series = [time_series.dropna() for time_series in all_time_series]

## Remove Stray Periods

In [ ]:
# features = [pv_info['sim_name_to_pv_name'][sim_name] for sim_name in model_info['model_in_list'] if pv_info['sim_name_to_pv_name'][sim_name] in time_series.columns]
# varying_features = [pvname for pvname in features if 'SOL' in pvname or 'QUAD' in pvname]
# outputs = ['OTRS:IN20:571:XRMS' ,'OTRS:IN20:571:YRMS']


In [ ]:
for time_series in all_time_series:
    plot_series(time_series)

In [ ]:
# plot_series(time_series,show=True)

In [ ]:
# mean_vals = pd.Series({
#     'CAMR:IN20:186:R_DIST': 453.46957443120476,
#     'Pulse_length': 1.8550514181818183,
#     'FBCK:BCI0:1:CHRG_S': 0.25,
#     'SOLN:IN20:121:BACT': 0.47600114995527293,
#     'QUAD:IN20:121:BACT': 0.014395751763488018,
#     'QUAD:IN20:122:BACT': 0.011783769915062949,
#     'ACCL:IN20:300:L0A_ADES': 58.0,
#     'ACCL:IN20:300:L0A_PDES': 0.0,
#     'ACCL:IN20:400:L0B_ADES': 70.0,
#     'ACCL:IN20:400:L0B_PDES': -2.5,
#     'QUAD:IN20:361:BACT': -3.3887000630248605,
#     'QUAD:IN20:371:BACT': 2.721178164561805,
#     'QUAD:IN20:425:BACT': -2.173085321374461,
#     'QUAD:IN20:441:BACT': -0.014265294503135854,
#     'QUAD:IN20:511:BACT': 2.8476903766544197,
#     'QUAD:IN20:525:BACT': -2.7030069765876372,
#     'OTRS:IN20:621:XRMS': 88.19116289583675,
#     'OTRS:IN20:621:YRMS': 73.2518868482159
# })

In [ ]:
# time_series = pd.read_pickle('data\calibration_2022-09-23_12_44_44.808775424-07_00__2022-09-23_23_59_58.905014016-07_00.pkl')
# time_series = time_series[0:30000]

In [ ]:
# for time_series in all_time_series:
#     print(len(time_series))

In [ ]:
# def remove_stray_periods(time_series, features, outputs):
#     sx = RegressionResidual(regressor=LinearRegression(), target=outputs[0]).fit_transform(time_series[features+outputs])
#     sy = RegressionResidual(regressor=LinearRegression(), target=outputs[1]).fit_transform(time_series[features+outputs])

#     sx = sx[abs(sx) < 20]
#     sy = sy[abs(sy) < 20]

#     combo_points = set(sx.index).intersection(set(sy.index))
#     test = time_series.loc[list(combo_points)].sort_index()
#     return test

# # test = remove_stray_periods(time_series, varying_features, outputs)
# # print(len(time_series))
# # print(len(test))

In [ ]:
# all_time_series = [df for df in [remove_stray_periods(time_series, varying_features, outputs) for time_series in all_time_series] if len(df) > 0]

In [ ]:
# for time_series in all_time_series:
#     print(len(time_series))

In [ ]:
for time_series in all_time_series:
    plot_series(time_series, save_dir='time_series_filtered')

## Reorder Features
Here we reorder the features into the order that the neural network model expects and set the timestamp as the index

In [ ]:
def reorder_features(time_series, features, outputs):
    time_series = time_series[features+outputs]
    return time_series

In [ ]:
features =  [pv_info['sim_name_to_pv_name'][sim_name] for sim_name in model_info['model_in_list'] if pv_info['sim_name_to_pv_name'][sim_name] in time_series.columns]
outputs =  [pv_info['sim_name_to_pv_name'][sim_name] for sim_name in model_info['model_out_list'] if pv_info['sim_name_to_pv_name'][sim_name] in time_series.columns]

In [ ]:
all_time_series = [reorder_features(time_series, features, outputs) for time_series in all_time_series]

## Save
Once all the processing is complete, we save the dataset with only the features and outputs of iterest to us during training. We also split the data into training, validation and test sets based on the month. 

For now we will use all of the data from December as validation / test data and the rest for training.

In [ ]:
training_data = []
validation_data = []
test_data = []
    
for time_series in all_time_series:
    date_str = get_date_from_df(time_series)
    date = datetime(*(int(i) for i in date_str.split('-')))
    if date.month == 12:
        if date.day == 9:
            test_data.append(time_series.drop_duplicates())
    elif date.day == 18 and date.month == 11:
        validation_data.append(time_series["2021-11-18 00:00:00-0800": "2021-11-18 09:00:00-0800"].drop_duplicates())
        training_data.append(time_series["2021-11-18 09:00:01-0800": "2021-11-19 00:00:00-0800"].drop_duplicates())
    else:
        training_data.append(time_series.drop_duplicates())

train_df = pd.concat(training_data).sort_index()
val_df = pd.concat(validation_data).sort_index()
test_df = pd.concat(test_data).sort_index()

In [ ]:
# check there's no overlap between the datasets

train_set = set(train_df.index)
val_set = set(val_df.index)
test_set = set(test_df.index)

print(len(train_set.intersection(val_set)) == 0)
print(len(val_set.intersection(test_set)) == 0)
print(len(train_set.intersection(test_set)) == 0)

In [ ]:
train_df.info()

In [ ]:
val_df.info()

In [ ]:
test_df.info()

In [ ]:
train_df.to_pickle('archive_data/train_df.pkl')
val_df.to_pickle('archive_data/val_df.pkl')
test_df.to_pickle('archive_data/test_df.pkl')

In [ ]:
test_df.info()